# 3- Conversational Agent

This notebook provides a prototype of the Conversational Agent, leveraging the in-memory embedding and vector storage capabilities.

## Set-up

In [12]:

from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

import os
import sys
from dotenv import load_dotenv

load_dotenv()

True

In [13]:
data_path = os.getenv("VECTOR_DB_DIR")

# Insantiates Embedding model
embedding_model_name = "thenlper/gte-base"
embedder = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
)

# Loads DB from disk
vectordb = Chroma(
    persist_directory=data_path,
    embedding_function=embedder
)

## Conversational Agent

In [14]:
qa_chain = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(),
    vectordb.as_retriever(search_kwargs={'k': 6}),
    return_source_documents=True
)

In [15]:
chat_history = []
while True:
    # this prints to the terminal, and waits to accept an input from the user
    query = input('Insert query: ')
    print()
    # give us a way to exit the script
    if query == "exit" or query == "quit" or query == "q":
        print('Exiting')
        sys.exit()
    # we pass in the query to the LLM, and print out the response. As well as
    # our query, the context of semantically relevant information from our
    # vector store will be passed in, as well as list of our chat history
    print('Query: ', query)
    result = qa_chain({'question': query, 'chat_history': chat_history})
    print('Answer: ' + result['answer'])
    # we build up the chat_history list, based on our question and response
    # from the LLM, and the script then returns to the start of the loop
    # and is again ready to accept user input.
    chat_history.append((query, result['answer']))
    print()

Query:  What services does KPMG offer?
Answer: KPMG offers a wide range of professional services including audit, tax, and advisory services. They provide services in areas such as financial statement audits, tax planning and compliance, risk management, management consulting, and more. For more detailed information about their services, please visit their official website at https://kpmg.com.

Exiting


SystemExit: 

c:\Users\juanr\Documents\Profesional\KPMG\technical_case\kworld_ir_chatbot\kworld_ai\.venv\lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## RetrievalQA and Similarity Search (Debugger)

In [9]:
#query = "What services does KPMG offer?"
query = "Does KPMG have offices in Canada?"
#query = "Get me a recipe for a lemon cupcake"
#query = "Who are KPMG's strategic alliance partners in technology?"
#query = "Who is Bill Thomas? What are his qualifications?"

qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=vectordb.as_retriever(search_kwargs={'k': 7}),
    return_source_documents=True
)

result = qa_chain({'query': query})
print(result['result'])

docs = vectordb.similarity_search(query)
print(docs)

 Yes, KPMG has offices in Canada.
[Document(page_content='KPMG in Canada', metadata={'source': 'https://kpmg.com/xx/en/home/industries/healthcare/covid-19-and-healthcare/connected-health/drinking-from-the-fire-hose.html'}), Document(page_content='KPMG in Canada', metadata={'source': 'https://kpmg.com/xx/en/home/industries/healthcare/covid-19-and-healthcare/connected-health/drinking-from-the-fire-hose.html'}), Document(page_content='KPMG in Canada', metadata={'source': 'https://kpmg.com/xx/en/home/industries/healthcare/covid-19-and-healthcare/connected-health/drinking-from-the-fire-hose.html'}), Document(page_content='KPMG in Canada', metadata={'source': 'https://kpmg.com/xx/en/home/industries/healthcare/covid-19-and-healthcare/connected-health/drinking-from-the-fire-hose.html'})]
